## Deliverable 2. Create a Customer Travel Destinations Map.

In [20]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Punta Arenas,-53.1500,-70.9167,37.51,87,20,12.66,CL,few clouds
1,1,Chokurdakh,70.6333,147.9167,34.74,87,100,16.96,RU,overcast clouds
2,2,Saskylakh,71.9167,114.0833,33.01,84,100,14.99,RU,overcast clouds
3,3,Vestmannaeyjar,63.4427,-20.2734,42.80,79,92,6.64,IS,overcast clouds
4,4,Busselton,-33.6500,115.3333,63.05,55,3,16.46,AU,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [56]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# N0 EMPTY ROWS :-)

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
    # Add NaN to empty row to easily drop row later if neccessary
hotel_df["Hotel Name"] = np.nan 
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kapaa,US,82.38,light rain,22.0752,-159.3190,NaN
11,Oussouye,SN,77.32,light rain,12.4850,-16.5469,NaN
12,Urucara,BR,76.68,overcast clouds,-2.5364,-57.7600,NaN
24,Adrar,MR,87.51,broken clouds,20.5022,-10.0711,NaN
26,Alenquer,BR,77.34,scattered clouds,-1.9417,-54.7383,NaN
36,Dehloran,IR,84.02,clear sky,32.6941,47.2679,NaN
38,Cabo San Lucas,MX,84.61,clear sky,22.8909,-109.9124,NaN
39,Samarai,PG,80.53,broken clouds,-10.6167,150.6667,NaN
40,Bubaque,GW,80.71,broken clouds,11.2833,-15.8333,NaN
45,Tiarei,PF,82.42,clear sky,-17.5333,-149.3333,NaN


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [67]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()



In [68]:
# Check for NaN
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [69]:
# Display DF
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Oussouye,SN,77.32,light rain,12.4850,-16.5469,Auberge du Routard
12,Urucara,BR,76.68,overcast clouds,-2.5364,-57.7600,Pousada Maria Bonita
26,Alenquer,BR,77.34,scattered clouds,-1.9417,-54.7383,Josilene Castro
36,Dehloran,IR,84.02,clear sky,32.6941,47.2679,کافه گیم استورم
38,Cabo San Lucas,MX,84.61,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
39,Samarai,PG,80.53,broken clouds,-10.6167,150.6667,Nuli Sapi
40,Bubaque,GW,80.71,broken clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
45,Tiarei,PF,82.42,clear sky,-17.5333,-149.3333,Le Rocher de Tahiti
49,Ha Tinh,VN,78.17,moderate rain,18.3333,105.9000,Phú Quý


In [72]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [75]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))